In [ ]:
from google.colab import drive
drive._mount('/content/drive')

In [ ]:
!pip install sentence-transformers

#**Importing packages**

In [ ]:
import pandas as pd
import numpy as np
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn import model_selection, naive_bayes
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import classification_report,confusion_matrix

#**English dataset**

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/homo-trans/splited data/mini project II/eng/eng_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/homo-trans/splited data/mini project II/eng/eng_test.csv')

In [ ]:
train.value_counts('category')

In [ ]:
test.value_counts('category')

#**Removing punctuations**

In [ ]:
def remove_punctuations(txt):
    text_nopunc="".join([c for c in txt if c not in string.punctuation])
    return text_nopunc

train['text']=train['text'].apply(lambda x: remove_punctuations(x))
test['text']=test['text'].apply(lambda x: remove_punctuations(x))

#**Encoding**

In [ ]:
Encoder = LabelEncoder()
train['category']=Encoder.fit_transform(train['category'])
test['category']=Encoder.fit_transform(test['category'])

#**BERT embedding**

In [ ]:
#function for selecting data for training
def model_select(train,test):
  X_train = train['text']
  X_test = test['text']
  y_train = train['category']
  y_test = test['category']
  return X_train, X_test, y_train, y_test

#from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = model_select(train,test)

In [ ]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
sbert_model = SentenceTransformer('nli-bert-large')  #nli-roberta-base-v2
sbert_model = SentenceTransformer('nli-roberta-base-v2')
f_train = sbert_model.encode(X_train)
f_test = sbert_model.encode(X_test)

#**Naive Bayes**

In [ ]:
naive_bayes = BernoulliNB()
# naive_bayes = MultinomialNB()
naive_bayes.fit(f_train, y_train)
pred_NB = naive_bayes.predict(f_test)

In [ ]:
print(confusion_matrix(y_test, pred_NB)) 
print(classification_report(y_test,pred_NB))

#**Support Vector Machine**

In [ ]:
# normal SVM
from sklearn.svm import SVC
SVM = SVC(C=10, kernel = 'linear', degree = 3, class_weight='balanced'   , gamma = 'auto')
X_Train_svm = SVM.fit(f_train, y_train)
pred_svm = SVM.predict(f_test)

In [ ]:
print(confusion_matrix(y_test,pred_svm)) 
print(classification_report(y_test, pred_svm))

#**Logistic Regression**

In [ ]:
lr = LogisticRegression(C=2,class_weight = 'balanced',max_iter=1000)
lr.fit(f_train,y_train)
pred_LR = lr.predict(f_test)

In [ ]:
print(confusion_matrix(y_test,pred_LR)) 
print(classification_report(y_test,pred_LR))

#**Random Forest**

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler(with_mean=False)
X_Train_RF = sc_X.fit_transform(f_train)
test_RF = sc_X.transform(f_test)

#Classifier
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=10, class_weight='balanced')
classifier.fit(X_Train_RF, y_train)

# Predicting the dev set 
Pred_RF = classifier.predict(test_RF)

In [ ]:
print(confusion_matrix(y_test,Pred_RF))
print(classification_report(y_test,Pred_RF))

#**Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
clf = clf.fit(f_train, y_train)
Pred_DT = clf.predict(f_test)

In [ ]:
print(confusion_matrix(y_test,Pred_DT))
print(classification_report(y_test,Pred_DT))

#**XGBoost**

In [ ]:

xgboost = XGBClassifier()
xgboost.fit(np.array(f_train), y_train)
pred_xg = xgboost.predict(f_test)

In [ ]:
print(confusion_matrix(y_test, pred_xg)) 
print(classification_report(y_test,pred_xg))